In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Load and prepare data
df["Date Reported"] = pd.to_datetime(df["Date Reported"])
df["Month"] = df["Date Reported"].dt.to_period("M").astype(str)

sankey_data = df.groupby(["Month", "Crime Type", "Status"]).size().reset_index(name="Count")

labels = list(pd.unique(sankey_data['Crime Type'].tolist() + sankey_data['Status'].tolist()))
label_indices = {label: i for i, label in enumerate(labels)}

months = sorted(sankey_data["Month"].unique())

# Beautiful cosmic palette
cosmic_palette = [
    (138, 43, 226),   # BlueViolet
    (0, 255, 255),    # Cyan
    (255, 105, 180),  # HotPink
    (30, 144, 255),   # DodgerBlue
    (0, 206, 209),    # DarkTurquoise
    (255, 165, 0),    # Orange
    (148, 0, 211),    # DarkViolet
    (255, 20, 147),   # DeepPink
    (135, 206, 250),  # LightSkyBlue
    (72, 61, 139)     # DarkSlateBlue
]

crime_types = sankey_data["Crime Type"].unique()
base_colors = {crime: cosmic_palette[i % len(cosmic_palette)] for i, crime in enumerate(crime_types)}

frames = []

all_sources = []
all_targets = []
all_values = []

bloom_steps = 5  # number of mini blooms within each month

for idx, month in enumerate(months):
    month_data = sankey_data[sankey_data["Month"] == month]

    month_sources = month_data['Crime Type'].map(label_indices).tolist()
    month_targets = month_data['Status'].map(label_indices).tolist()
    month_values = month_data['Count'].tolist()

    for bloom in range(1, bloom_steps + 1):
        bloom_sources = all_sources + month_sources
        bloom_targets = all_targets + month_targets
        bloom_values = all_values + month_values

        frame_colors = []
        for i in range(len(bloom_sources)):
            source_idx = bloom_sources[i]
            if source_idx < len(labels):
                source_label = labels[source_idx]
            else:
                source_label = None

            if source_label and source_label in base_colors:
                r, g, b = base_colors[source_label]
            else:
                r, g, b = (0,191,255)

            alpha = 0.3 + 0.7 * (bloom / bloom_steps)
            frame_colors.append(f"rgba({int(r)},{int(g)},{int(b)},{alpha:.2f})")

        frame = go.Frame(
            data=[go.Sankey(
                arrangement="snap",
                node=dict(
                    pad=20,
                    thickness=30,
                    line=dict(color="black", width=0.5),
                    label=labels,
                    color="rgba(20,20,20,0.95)"
                ),
                link=dict(
                    source=bloom_sources,
                    target=bloom_targets,
                    value=bloom_values,
                    color=frame_colors
                )
            )],
            name=f"{month} - bloom {bloom}"
        )
        frames.append(frame)

    all_sources += month_sources
    all_targets += month_targets
    all_values += month_values

# Initialize figure
initial_sources = sankey_data[sankey_data["Month"] == months[0]]['Crime Type'].map(label_indices).tolist()
initial_targets = sankey_data[sankey_data["Month"] == months[0]]['Status'].map(label_indices).tolist()
initial_values = sankey_data[sankey_data["Month"] == months[0]]['Count'].tolist()

initial_colors = []
for src in initial_sources:
    label = labels[src]
    if label in base_colors:
        r, g, b = base_colors[label]
    else:
        r, g, b = (0,191,255)
    initial_colors.append(f"rgba({int(r)},{int(g)},{int(b)},0.3)")

fig = go.Figure(
    data=[go.Sankey(
        arrangement="snap",
        node=dict(
            pad=20,
            thickness=30,
            line=dict(color="black", width=0.5),
            label=labels,
            color="rgba(20,20,20,0.95)"
        ),
        link=dict(
            source=initial_sources,
            target=initial_targets,
            value=initial_values,
            color=initial_colors
        )
    )],
    frames=frames
)

fig.update_layout(
    font=dict(size=14, color='white'),
    plot_bgcolor='black',
    paper_bgcolor='black',
    height=750,
    width=1200,
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(label="▶️ Bloom", method="animate", args=[None, {
            "frame": {"duration": 1200, "redraw": True},
            "fromcurrent": True,
            "transition": {"duration": 1000, "easing": "linear"}
        }])]
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f"{month} - bloom {bloom}"], {"frame": {"duration": 1200, "redraw": True}, "mode": "immediate"}], label=f"{month} - {bloom}") for month in months for bloom in range(1, bloom_steps+1)],
        transition={"duration": 1000},
        x=0.1,
        len=0.8
    )]
)

fig.show()


<ipython-input-12-2585c65ca4ca>:11: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.



"Animated Sankey diagram showing the monthly evolution of Crime Types flowing into their Status outcomes, using real campus incident data. Each bloom represents the emergence of new cases, color-coded by crime category, highlighting the dynamic progression over time."

In [ ]:
# Animated Bar chart showcasing Crime TypeFrequency Over Time
import pandas as pd
import plotly.express as px

# Make sure Date Reported is datetime
df["Date Reported"] = pd.to_datetime(df["Date Reported"])

# Extract month
df["Month"] = df["Date Reported"].dt.to_period("M").astype(str)

# Group by Crime Type and Month to get frequency
bar_data = df.groupby(["Month", "Crime Type"]).size().reset_index(name="Frequency")

# Create animated bar chart
fig = px.bar(
    bar_data,
    x="Crime Type",
    y="Frequency",
    color="Crime Type",
    animation_frame="Month",
    animation_group="Crime Type",
    title="Animated Bar Chart: Crime Type Frequency Over Time",
    range_y=[0, bar_data["Frequency"].max() + 10],
    template="plotly_white"
)

# Optional: Sort bars within each frame
fig.update_layout(xaxis={'categoryorder':'total descending'})

fig.show()

# Animated Crime Frequency Timeline
import pandas as pd
import plotly.express as px

# Ensure datetime format
df["Date Reported"] = pd.to_datetime(df["Date Reported"])

# Group by Date and Crime Type
time_data = df.groupby(["Date Reported", "Crime Type"]).size().reset_index(name="Frequency")

# Create animated time series line chart
fig = px.line(
    time_data,
    x="Date Reported",
    y="Frequency",
    color="Crime Type",
    title="Animated Crime Frequency Timeline",
    labels={"Date Reported": "Date", "Frequency": "Number of Crimes"},
    animation_frame="Crime Type",   # Animate through different Crime Types
    template="simple_white"
)

fig.update_layout(
    font=dict(size=14),
    title_font=dict(size=22),
    plot_bgcolor='white',
    showlegend=True,
    hovermode="x unified"
)

fig.show()

#  Crime Frequency Over Time (Daily Trend)

# Group by Date alone
daily_data = df.groupby("Date Reported").size().reset_index(name="Frequency")

# Simple Line Plot
fig = px.line(
    daily_data,
    x="Date Reported",
    y="Frequency",
    title=" Crime Frequency Over Time (Daily Trend)",
    labels={"Date Reported": "Date", "Frequency": "Number of Crimes"},
    template="plotly_white"
)

fig.update_traces(mode='lines+markers')  # Adds points to the line
fig.update_layout(
    font=dict(size=14),
    title_font=dict(size=22),
    plot_bgcolor='white',
    hovermode="x unified"
)

fig.show()


NameError: name 'df' is not defined